11756 - Storage and Data Analysis - Exercise 5: XML to Database

Amelia Yolanda & June Rose Manuzon

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import sqlite3

In [2]:
# Parse the XML file
tree = ET.parse('/content/AllPeople.xml') # Upload file to folder in Colab runtime

root = tree.getroot()

In [3]:
# Extract data from XML and create a DataFrame
data = []
for item in root.findall('person'):
    gender = item.find('gender').text
    name = item.find('name').text
    altername = item.find('alter').text
    surname1 = item.find('surname1').text
    surname2 = item.find('surname2').text
    birth = item.find('birth').text
    fathername = item.find('fathername').text
    fathersurname1 = item.find('fathersurname1').text
    fathersurname2 = item.find('fathersurname2').text
    mothername = item.find('mothername').text
    mothersurname1 = item.find('mothersurname1').text
    mothersurname2 = item.find('mothersurname2').text
    father_grandfathername = item.find('father_grandfathername').text
    father_grandmothername = item.find('father_grandmothername').text
    mother_grandfathername = item.find('mother_grandfathername').text
    mother_grandmothername = item.find('mother_grandmothername').text
    data.append({
        'gender': gender,
        'name': name,
        'altername': altername,
        'surname1': surname1,
        'surname2': surname2,
        'birth': birth,
        'fathername': fathername,
        'fathersurname1': fathersurname1,
        'fathersurname2': fathersurname2,
        'mothername': mothername,
        'mothersurname1': mothersurname1,
        'mothersurname2': mothersurname2,
        'father_grandfathername': father_grandfathername,
        'father_grandmothername': father_grandmothername,
        'mother_grandfathername': mother_grandfathername,
        'mother_grandmothername': mother_grandmothername,
        })

# Add data from XML to dataframe
df = pd.DataFrame(data)

In [4]:
df.tail()

,gender,name,altername,surname1,surname2,birth,fathername,fathersurname1,fathersurname2,mothername,mothersurname1,mothersurname2,father_grandfathername,father_grandmothername,mother_grandfathername,mother_grandmothername
478,D,Joana,"Ana, Maria",Ferrer,Tortella,1820-11-21,Josef,Ferrer,Ferregut,Francisca,Tortella,Bertran,Juan,Margarita,Antonio,Juana
479,H,Martin,"Luis, Ramon",Coll,Ferrer,1820-11-28,Martin,Coll,Torrens,Antonia,Ferrer,Payeras,Juan,Magdalena,Jayme,Juana
480,H,Gabriel,"Josef, Juan, Mariano",Matheu,Figuerola,1820-11-28,Ignacio,Matheu,Marcant,Francisca,Figuerola,Martorell,Gabriel,Maria,Juan,Juana
481,D,Juana,"Ana, Maria",Vallespir,Garau,1820-11-29,Antonio,Vallespir,Fiol,Cathalina,Garau,Bertran,Pedro,Juana,Jayme,Antonia
482,H,Antonio,"Silvestre, Mariano",Martorell,Llobera,1820-11-31,Jayme,Martorell,Morro,Francisca,Llobera,Llompard,Miguel,Juana,Antonio,Cathalina


In [5]:
# temp, option to use sqlalchemy

from sqlalchemy import create_engine

# SQLite example connection string
engine = create_engine('sqlite:///11756.db')

# Write the DataFrame to the database
df.to_sql('People', engine, index=False, if_exists='replace')

483

In [6]:
# Define functions to store dataframe to sql through notebook
# credit source code: https://colab.research.google.com/github/stephenleo/medium-python-hacks/blob/main/02_sql_on_colab/main.ipynb

def pd_to_sqlDB(input_df: pd.DataFrame, table_name: str, db_name: str = 'default.db') -> None:
  # Step 1: Setup local logging
  import logging
  logging.basicConfig(level=logging.INFO,
                      format='%(asctime)s %(levelname)s: %(message)s',
                      datefmt='%Y-%m-%d %H:%M:%S')

  # Step 2: Find columns in the dataframe
  cols = input_df.columns
  cols_string = ','.join(cols)
  val_wildcard_string = ','.join(['?'] * len(cols))

  # Step 3: Connect to a DB file if it exists, else crete a new file
  con = sqlite3.connect(db_name)
  cur = con.cursor()
  logging.info(f'SQL DB {db_name} created')

  # Step 3.5: Delete Existing Table
  sql_string = f"""DROP TABLE IF EXISTS {table_name};"""
  cur.execute(sql_string)

  # Step 4: Create Table
  sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
  print(sql_string)
  cur.execute(sql_string)
  logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

  # Step 5: Upload the dataframe
  rows_to_upload = input_df.to_dict(orient='split')['data']
  sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""
  cur.executemany(sql_string, rows_to_upload)
  logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')

  # Step 6: Commit the changes and close the connection
  con.commit()
  con.close()

In [7]:
pd_to_sqlDB(df, table_name='people', db_name='default.db')

CREATE TABLE people (gender,name,altername,surname1,surname2,birth,fathername,fathersurname1,fathersurname2,mothername,mothersurname1,mothersurname2,father_grandfathername,father_grandmothername,mother_grandfathername,mother_grandmothername);


In [8]:
# Download database (to access in DBeaver)
from google.colab import files

files.download('default.db') # File can be downloaded from side panel

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
# Define functions to retrieve query and store to dataframe >> to be done in DBeaver

def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe

    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.

    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

In [10]:
# Write the SQL query in a string variable
sql_query_string = """
    SELECT DISTINCT(name), COUNT(name)
    FROM people
    GROUP BY name
"""

# Exectue the SQL query and store in dataframe
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df.head()

,name,COUNT(name)
0,Ana,4
1,Andres,2
2,Anna,3
3,Antonia,33
4,Antonio,24
